In [1]:
file1 = "../remote/Silicon_etch.off"
# file1 = "../remote/Nitride_etch.obj"
file2 = "../initial_struct_600_600.obj"

In [ ]:
import open3d as o3d
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import os

def load_off_as_pointcloud(file_path, num_points=10000):
    mesh = trimesh.load(file_path)
    sampled_points = mesh.sample(num_points)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(sampled_points)
    return pcd

def compute_rms(source, target):
    distances = source.compute_point_cloud_distance(target)
    return np.sqrt(np.mean(np.square(distances)))

def run_icp(source, target, threshold=0.05):
    trans_init = np.eye(4)
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    return icp_result.transformation, icp_result

def visualize_alignment(pcd1, pcd2, transformed_pcd1, save_path="alignment.png"):
    pcd1.paint_uniform_color([1, 0, 0])  # red
    pcd2.paint_uniform_color([0, 1, 0])  # green
    transformed_pcd1.paint_uniform_color([0, 0, 1])  # blue

    o3d.visualization.draw_geometries([pcd2, transformed_pcd1])

    # Save image
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    vis.add_geometry(pcd2)
    vis.add_geometry(transformed_pcd1)
    vis.poll_events()
    vis.update_renderer()
    vis.capture_screen_image(save_path)
    vis.destroy_window()

def generate_report(before_rms, after_rms, save_path="report.txt"):
    with open(save_path, 'w') as f:
        f.write("ICP Alignment Report\n")
        f.write("=====================\n")
        f.write(f"RMS Error Before ICP: {before_rms:.6f}\n")
        f.write(f"RMS Error After ICP : {after_rms:.6f}\n")
        f.write(f"\nSee 'alignment.png' for visual comparison.\n")

def main(file1, file2):
    print("Loading point clouds from OFF files...")
    source = load_off_as_pointcloud(file1)
    target = load_off_as_pointcloud(file2)

    print("Computing initial RMS error...")
    rms_before = compute_rms(source, target)

    print("Running ICP registration...")
    transformation, icp_result = run_icp(source, target)

    transformed_source = source.transform(transformation)
    rms_after = compute_rms(source, target)

    print(f"RMS Before ICP: {rms_before:.6f}")
    print(f"RMS After ICP : {rms_after:.6f}")

    print("Visualizing and saving alignment result...")
    visualize_alignment(source, target, source, "alignment.png")

    print("Generating report...")
    generate_report(rms_before, rms_after)

    print("Done. See 'alignment.png' and 'report.txt'.")

if __name__ == "__main__":
    # Replace with your actual OFF file paths
    main(file1, file2)


Loading point clouds from OFF files...
Computing initial RMS error...
Running ICP registration...
RMS Before ICP: 11.968453
RMS After ICP : 11.968453
Visualizing and saving alignment result...
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Generating report...
Done. See 'alignment.png' and 'report.txt'.


: 